In [1]:
import os 
import math
import random
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from numpy import count_nonzero

dir_ = '../data/'
group_dir_ = '../data/groups/random'

In [ ]:
file_name = 'normalized_to_rating_filter_track_5_user_100.csv'
svd = pd.read_pickle(os.path.join(dir_, 'prediction_svd_top_N_' + file_name[:-3] + 'pkl'))
test = pd.read_pickle(os.path.join(dir_, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])

file_name = 'normalized_popularity_filter_track_5_user_100.pkl'
pop_count = pd.read_pickle(os.path.join(dir_, file_name))

num_user = len(svd['uid'].unique())
print(num_user, len(svd))

In [ ]:
groups_n =[]
group_sizes = ['2', '3', '4', '5']
for i in range(len(group_sizes)):
    groups = np.loadtxt(os.path.join(group_dir_, group_sizes[i] + '.csv'), delimiter=',')
    groups_n.append(groups)

In [ ]:
for i in groups_n:
    print(len(i[0]))

In [ ]:
svd['count'] = 1
svd[:5]

In [ ]:
pop_count = pop_count.sort_values(by=['count'],  ascending=False)
pop_count[:10]

In [7]:
# Linear
size = int(len(pop_rating) // 5)
pop_normalized = pd.DataFrame()
for i in range(5):
    if i == 0:
        pop = pop_count[:size].copy()
    elif i == 4:
        pop = pop_count[size*4:].copy()
    else:
        pop = pop_count[size*i:size*(i+1)].copy()
    pop['rating'] = (5 - i)
    pop_normalized = pop_normalized.append(pop)
pop_normalized = pop_normalized[['tid', 'rating']]


In [8]:
# min_max
max_rating = pop_count.iloc[0]['count']
pop_normalized = pop_count.copy()
pop_normalized = pop_normalized[['tid', 'count']]
pop_normalized['rating'] = pop_normalized['count']
pop_normalized['rating'] /= max_rating
pop_normalized['rating'] *= 5
pop_normalized[:10]

,tid,count,rating
70,70,17557,5.000000
1102,1102,6926,1.972433
83,83,6571,1.871333
13496,13496,6299,1.793871
952,952,6234,1.775360
2519,2519,6001,1.709005
4008,4008,5676,1.616449
5521,5521,5658,1.611323
3900,3900,5627,1.602495
2241,2241,5571,1.586547


In [9]:
lambdas = []
for i in range(11):
    lambdas.append(i/10)
lambdas

[0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

In [10]:
for lambda_ in tqdm(lambdas):
    top_n_size = 4
    top_n_lists_g = []
    for groups in groups_n:

        group_size = len(groups[0])
        top_n_lists = []

        for group in groups:
            rating_table = pd.DataFrame() 
            for member in group:
                prediction = svd[svd['uid'] == member].copy()
                if len(rating_table) == 0:
                    rating_table = prediction
                else:
                    rating_table = rating_table.set_index('tid').add(prediction.set_index('tid'), fill_value=0).reset_index()
            rating_table['rating'] /= group_size 
            rating_table['rating'] *= lambda_
            pop = pop_normalized.copy()
            pop['rating'] *= (1 - lambda_)
            rating_table = rating_table.set_index('tid').add(pop.set_index('tid'), fill_value=0).reset_index()
    #         rating_table = rating_table[rating_table['count'] == group_size]
            rating_table = rating_table.sort_values(by=['rating'],  ascending=False)
            rating_table = rating_table[:top_n_size]
            top_n_list = []
            for _, row in rating_table.iterrows():
                top_n_list.append(row[0])
            top_n_lists.append(top_n_list)
        top_n_lists_g.append(top_n_lists)   
        
#     print('lambda = ' + str(lambda_))
    threshold = 0.0
    avg_precisions = []
    for i in range(len(groups_n)):
        groups = groups_n[i]
        top_n_lists = top_n_lists_g[i]

        precisions = []
        for j in (range(len(groups))):

            group = groups[j]
            top_n_list = top_n_lists[j]

            high_rating = 0

            for k in range(top_n_size):
                for l in range(len(group)):
                    uid = group[l]
                    tid = top_n_list[k]
                    t = test[test['uid'] == uid]
                    t = t[t['tid'] == tid]
                    if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                        high_rating += 1
                        break

            precision = high_rating / top_n_size
            precisions.append(precision)

        avg_precision = 0
        for precision in precisions:
            avg_precision += precision
        avg_precision /= len(precisions)
        avg_precisions.append(avg_precision)
    print(avg_precisions)
#     print(sum(avg_precisions) / len(avg_precisions))
#     print('----------------------------------------') 

[0.2347689075630252, 0.3225552050473186, 0.3949579831932773, 0.4868421052631579]
[0.2494747899159664, 0.3359621451104101, 0.4023109243697479, 0.49605263157894736]
[0.25262605042016806, 0.34936908517350157, 0.41281512605042014, 0.5039473684210526]
[0.2610294117647059, 0.35331230283911674, 0.4170168067226891, 0.5039473684210526]
[0.2762605042016807, 0.36514195583596215, 0.4222689075630252, 0.5039473684210526]
[0.2977941176470588, 0.3872239747634069, 0.43487394957983194, 0.525]
[0.32090336134453784, 0.4148264984227129, 0.4695378151260504, 0.5434210526315789]
[0.35084033613445376, 0.43296529968454256, 0.4947478991596639, 0.5552631578947368]
[0.3707983193277311, 0.444006309148265, 0.509453781512605, 0.575]
[0.39810924369747897, 0.47397476340694006, 0.5273109243697479, 0.5947368421052631]
[0.36869747899159666, 0.42665615141955837, 0.47058823529411764, 0.48026315789473684]



In [195]:
alpha = 0.5
threshold = 0.0
for i in tqdm(range(len(groups_n))):
    
    groups = groups_n[i]
    top_n_lists = top_n_lists_g[i]
    
    satisfactions = []
    for j in (range(len(groups))):
        
        group = groups[j]
        top_n_list = top_n_lists[j]
        
        high_ratings_t = np.zeros(top_n_size)
        high_ratings_u = np.zeros(len(group))
        
        for k in range(top_n_size):
            for l in range(len(group)):
                uid = group[l]
                tid = top_n_list[k]
                t = test[test['uid'] == uid]
                t = t[t['tid'] == tid]
                if len(t) > 0 and t.iloc[0]['rating'] > threshold:
                    high_ratings_t[k] += 1
                    high_ratings_u[l] += 1
        
#         satisfaction_t = 0
#         for k in range(top_n_size):
#             satisfaction_t += math.log(high_ratings_t[k]+1, 10) / math.log(len(group)+1, 10)
#         satisfaction_t /= top_n_size
        
#         satisfaction_u = 0
#         for k in range(len(group)):
#             satisfaction_u += math.log(high_ratings_u[k]+1, 10) / math.log(top_n_size+1, 10)
#         satisfaction_u /= len(group)
        satisfaction_t = math.log(max(high_ratings_t)+1, 10) / math.log(len(group)+1, 10)
        satisfaction_u = math.log(min(high_ratings_u)+1, 10) / math.log(top_n_size+1, 10)
        
        satisfaction = alpha*satisfaction_t + (1-alpha)*satisfaction_u
        satisfactions.append(satisfaction)
    
    avg_satisfaction = 0
    for satisfaction in satisfactions:
        avg_satisfaction += satisfaction
    avg_satisfaction /= len(satisfactions)
    print(avg_satisfaction)
        

0.4384689835799772
0.34093950544457624
0.3358646172870294
0.33769451038646087

